# Homework 3 - Programming Section Notebook

## Context

Given a dataset of 5875 patients showcasing different medical metrics related to Parkinson's diagnostic.
We must create a predictor for this disease that scores the likelihood according to the Unified Parkinson's Disease Rating Scale

In [29]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

dataset_file_path = "parkinsons.csv"

data_df = pd.read_csv(dataset_file_path)

X = data_df.drop("target", axis=1)
y = data_df["target"]

## 5 - Mean Absolute Error across different regression models (Linear, MLP no activation, MLP with ReLU activation)

## 6 - Importance of activation functions in MLP

## 7 - Mean Absolute Error across different hyperparameters combinations for a MLP regressor